<a href="https://colab.research.google.com/github/saktiworkstation/GNNs-LLMs-Research/blob/main/Simple_GCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install torch torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 80.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12


In [3]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.loader import DataLoader

import itertools
import numpy as np
import random

In [4]:
# Dataset Cora
dataset = Planetoid(root='data/Cora', name='Cora')
data = dataset[0]  # Ambil graph tunggal di dataset

Processing...
Done!


In [5]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, dropout):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
        self.dropout = dropout

    def forward(self, x, edge_index):
        # Layer 1
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)

        # Layer 2
        x = self.conv2(x, edge_index)
        return x

In [6]:
def train(model, data, optimizer):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return float(loss)


def test(model, data):
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)

    # Hitung akurasi
    accs = []
    for mask_name in ['train_mask', 'val_mask', 'test_mask']:
        mask = data[mask_name]
        correct = pred[mask].eq(data.y[mask]).sum().item()
        acc = correct / mask.sum().item()
        accs.append(acc)
    return accs  # [acc_train, acc_val, acc_test]

In [7]:
# Definisikan daftar hyperparameter untuk grid search
learning_rates = [0.01, 0.005]
hidden_channels_list = [16, 32]
dropouts = [0.3, 0.5]
epochs = 200  # jumlah epoch training

best_val_acc = 0.0
best_test_acc = 0.0
best_params = None

for lr, hidden_channels, dropout in itertools.product(learning_rates, hidden_channels_list, dropouts):
    # Inisialisasi model
    model = GCN(
        in_channels=dataset.num_features,
        hidden_channels=hidden_channels,
        out_channels=dataset.num_classes,
        dropout=dropout
    )

    # Gunakan Adam sebagai optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=5e-4)

    # Training
    for epoch in range(1, epochs+1):
        loss = train(model, data, optimizer)

    # Evaluasi
    train_acc, val_acc, test_acc = test(model, data)

    # Cek apakah model ini lebih baik di val_acc
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_test_acc = test_acc
        best_params = (lr, hidden_channels, dropout)

    print(f"[lr={lr}, hidden={hidden_channels}, dropout={dropout}] "
          f"Val Acc: {val_acc:.4f} | Test Acc: {test_acc:.4f}")

print("\n===== HASIL TERBAIK =====")
print(f"Best Params   : lr={best_params[0]}, hidden={best_params[1]}, dropout={best_params[2]}")
print(f"Val Acc       : {best_val_acc:.4f}")
print(f"Test Acc      : {best_test_acc:.4f}")

[lr=0.01, hidden=16, dropout=0.3] Val Acc: 0.7700 | Test Acc: 0.8080
[lr=0.01, hidden=16, dropout=0.5] Val Acc: 0.7620 | Test Acc: 0.7990
[lr=0.01, hidden=32, dropout=0.3] Val Acc: 0.7660 | Test Acc: 0.8020
[lr=0.01, hidden=32, dropout=0.5] Val Acc: 0.7580 | Test Acc: 0.7980
[lr=0.005, hidden=16, dropout=0.3] Val Acc: 0.7760 | Test Acc: 0.8050
[lr=0.005, hidden=16, dropout=0.5] Val Acc: 0.7700 | Test Acc: 0.7950
[lr=0.005, hidden=32, dropout=0.3] Val Acc: 0.7680 | Test Acc: 0.8080
[lr=0.005, hidden=32, dropout=0.5] Val Acc: 0.7800 | Test Acc: 0.8020

===== HASIL TERBAIK =====
Best Params   : lr=0.005, hidden=32, dropout=0.5
Val Acc       : 0.7800
Test Acc      : 0.8020
